# Adding an ExoGENI Stitched VLAN to an Existing Network  

## Overview:

This tutorial will guide you through setting up a stitched a network to an ExoGENI slice. 

After completing this tutorial you should be able to:

1. Stitch Chameleon networks to ExoGENI

## Prerequisites:

This tutorial assumes you have a Chameleon account and basic experience logging into and using Chameleon and basic use of Chameleon networks and isolated VLANs. 

Additional information:

- Getting started tutorial: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html
- Networks and Isolated VLANs:  https://chameleoncloud.readthedocs.io/en/latest/technical/networks.html
- Setting up the CLI: https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html

## Background:

### Tutorial:

#### Setup Environment:

In [15]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@UC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-chameleon-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/${SSH_KEY_NAME}


# Add Chameleon custom blazar client
#sudo pip install -e git+https://github.com/ChameleonCloud/python-blazarclient.git@allocatable-vlans#egg=python-blazarclient


#### Set/Find the Existing Network Attributes  

In [16]:
# Set the existing network name. 
EXISTING_NETWORK_NAME=${USERNAME}"Network"

# Set the additional VLAN's OpenStack network name
NEW_NETWORK_NAME=${EXISTING_NETWORK_NAME}EG1

# Set the additional VLAN's lease name 
NEW_NETWORK_LEASE_NAME=${NEW_NETWORK_NAME}-lease

# Get existing network's description
EXISTING_NETWORK_DESCRIPTION=`openstack network show  --format value -c description ${EXISTING_NETWORK_NAME}`

echo NEW_NETWORK_NAME: $NEW_NETWORK_NAME
echo EXISTING_NETWORK_DESCRIPTION: $EXISTING_NETWORK_DESCRIPTION

NEW_NETWORK_NAME: pruthNetworkEG1
EXISTING_NETWORK_DESCRIPTION: OFController=192.5.87.113:6653,VSwitchName=pruthSwitch


#### Add the ExoGENI VLAN Switch

In [18]:
echo Creating network ${NEW_NETWORK_NAME}
blazar lease-create \
   --reservation resource_type="network",network_name="${NEW_NETWORK_NAME}",network_description="${EXISTING_NETWORK_DESCRIPTION}" \
   --start-date "2019-02-11 16:43" \
   --end-date "2019-02-12 12:00" \
   ${NEW_NETWORK_LEASE_NAME}

Creating network pruthNetworkEG1
Matches: ('resource_type=network,network_name=pruthNetworkEG1', 'network_description', 'OFController=192.5.87.113:6653,VSwitchName=pruthSwitch')
Matches: ('resource_type=network', 'network_name', 'pruthNetworkEG1')
Matches: (None, 'resource_type', 'network')
Created a new lease:
+--------------+------------------------------------------------------------+
| Field        | Value                                                      |
+--------------+------------------------------------------------------------+
| created_at   | 2019-02-11 16:41:33                                        |
| degraded     | False                                                      |
| end_date     | 2019-02-12T12:00:00.000000                                 |
| events       | {                                                          |
|              |     "status": "UNDONE",                                    |
|              |     "lease_id": "7e45d130-7a6b-470d-adbb-07d5b

After the reservation has started you can get the new uplink vlan tag

In [19]:
echo Finding Uplink VLAN
NEW_UPLINK_VLAN=`openstack network show -c provider:segmentation_id -f value ${NEW_NETWORK_NAME}`
echo Primary uplink VLAN and port ID: $NEW_UPLINK_VLAN 

Finding Uplink VLAN
Error while executing command: No Network found for pruthNetworkEG1
Primary uplink VLAN and port ID:


# Please Cleanup Your Resources!!!

Unlink and delete all pieces of the OpenFlow network.

In [14]:
echo Deleting network ${NEW_NETWORK_NAME}
blazar lease-delete ${NEW_NETWORK_LEASE_NAME}

Deleting network pruthNetworkEG1
There are more than one appropriate resources for the name 'pruthNetworkEG1-lease' and type 'lease'


: 1